In [1]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from pyspark.sql.functions import date_format
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


In [2]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "10g")
    .getOrCreate()
)

22/10/05 01:12:40 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/05 01:12:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 01:12:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read in data from ETL.py file
%run '../scripts/ETL.py' '../scripts/paths.json'
final_join3.limit(5)

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/05 01:12:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/05 01:13:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021
Egestas Nunc Asso...,11121775571,digital goods: bo...,6.58,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,11121775571,11.28829564583802,2bd2a61d-72e5-42d...,2021-08-20,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Morbi Accumsan In...,19618998054,tent and aWning s...,1.52,c,Christopher Rodri...,30554 Evans Strea...,NSW,Male,19618998054,62.90176609196828,3582b1f8-4577-403...,2021-05-16,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Dolor Egestas PC,94472466107,"cable, satellite,...",6.23,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,94472466107,172.15375126873164,cb05d49f-c2fa-453...,2021-07-22,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Urna Justo Indust...,31472801314,music shops - mus...,6.56,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,31472801314,0.4894787650356477,aeec15c1-67e8-4cb...,2021-05-18,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Sem Pellentesq...,35424691626,"computers, comput...",3.9,b,Christopher Rodri...,30554 Evans Strea...,NSW,Male,35424691626,7.360217018778133,9df473ba-102d-461...,2021-07-04,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694


In [4]:
final_join3.count()

10540181

In [5]:
tagged_merchants = pd.read_csv("../data/curated/tagged_merchants.csv")
tagged_merchants = tagged_merchants.iloc[:,1:]
tagged_merchants.drop(['tags', 'name', 'cleaned_tags', 'store_type'], axis=1, inplace=True)
tagged_merchants.to_parquet("../data/curated/tagged_merchants.parquet")
tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

In [6]:
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)

In [7]:
tagged_merchants_sdf.show(5)

+-------------------+--------------------+
|tagged_merchant_abn|            category|
+-------------------+--------------------+
|        10023283211|           Furniture|
|        10142254217|         Electronics|
|        10165489824|        Toys and DIY|
|        10187291046|        Toys and DIY|
|        10192359162|Books, Stationary...|
+-------------------+--------------------+
only showing top 5 rows



In [8]:
final_join3.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
INNER JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")

joint = joint.drop('tagged_merchant_abn')

In [9]:
joint.count()

10109254

In [10]:
joint.createOrReplaceTempView("group")

a = spark.sql(""" 

SELECT *, (dollar_value - take_rate) AS total_earning
FROM group
""")

In [11]:
# Extracting the year, month, day from the timestamp
from pyspark.sql.functions import year, month

a = a.withColumn('Year', year(a.order_datetime))
a = a.withColumn('Month',month(a.order_datetime))


In [12]:
a.show(5)

+--------------------+------------+--------------------+---------+--------------+--------------------+--------------------+-----+------+------------------+------------------+--------------------+--------------+-------+-----------+---------+---------+---------+-------------------+----------------+-----------+-------------+-------------+----------+---------------+---------------+---------------+--------------------+------------------+----+-----+
|       merchant_name|merchant_abn|          categories|take_rate|revenue_levels|                name|             address|state|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|postcodes|  int_sa2| SA2_code|           SA2_name|income_2018-2019|total_males|total_females|total_persons|state_code|     state_name|population_2020|population_2021|            category|     total_earning|Year|Month|
+--------------------+------------+--------------------+---------+--------------+--------------------+--

In [13]:
a = a.drop('merchant_abn', 'categories','name', 'address', 'trans_merchant_abn', 'order_id','order_datetime','user_id','consumer_id','int_sa2',
'SA2_name','state_code','state_name','population_2020', 'population_2021','total_earning')

In [14]:
 
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
a.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in a.columns]
   ).show()

+-------------+---------+--------------+-----+------+------------+---------+--------+----------------+-----------+-------------+-------------+--------+----+-----+
|merchant_name|take_rate|revenue_levels|state|gender|dollar_value|postcodes|SA2_code|income_2018-2019|total_males|total_females|total_persons|category|Year|Month|
+-------------+---------+--------------+-----+------+------------+---------+--------+----------------+-----------+-------------+-------------+--------+----+-----+
|            0|        0|             0|    0|     0|           0|        0|       0|               0|          0|            0|            0|       0|   0|    0|
+-------------+---------+--------------+-----+------+------------+---------+--------+----------------+-----------+-------------+-------------+--------+----+-----+



In [15]:
a.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- postcodes: string (nullable = true)
 |-- SA2_code: long (nullable = true)
 |-- income_2018-2019: long (nullable = true)
 |-- total_males: long (nullable = true)
 |-- total_females: long (nullable = true)
 |-- total_persons: long (nullable = true)
 |-- category: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



In [16]:
a.createOrReplaceTempView("agg")

male = spark.sql(""" 

SELECT CONCAT(merchant_name, SA2_code, Year, Month) AS m_name, COUNT(gender) as males
FROM agg
WHERE gender = 'Male'
GROUP BY merchant_name, SA2_code, Year, Month
""")

male.show(5)

female = spark.sql(""" 

SELECT CONCAT(merchant_name, SA2_code, Year, Month) AS f_name, COUNT(gender) as females
FROM agg
WHERE gender = 'Female'
GROUP BY merchant_name, SA2_code, Year, Month
""")
female.show(5)

+--------------------+-----+
|              m_name|males|
+--------------------+-----+
|Semper Tellus PC1...|    2|
|Est Nunc Consulti...|   11|
|Ipsum Primis In I...|    2|
|Euismod In LLC601...|    1|
|Leo In Consulting...|    2|
+--------------------+-----+
only showing top 5 rows



+--------------------+-------+
|              f_name|females|
+--------------------+-------+
|Quis Tristique Lt...|      1|
|Nunc In Industrie...|      2|
|Nunc Sit LLC10902...|      2|
|Leo In Consulting...|      5|
|Risus Donec Assoc...|      1|
+--------------------+-------+
only showing top 5 rows



In [17]:
a.show(2)

+--------------------+---------+--------------+-----+------+-----------------+---------+---------+----------------+-----------+-------------+-------------+--------------------+----+-----+
|       merchant_name|take_rate|revenue_levels|state|gender|     dollar_value|postcodes| SA2_code|income_2018-2019|total_males|total_females|total_persons|            category|Year|Month|
+--------------------+---------+--------------+-----+------+-----------------+---------+---------+----------------+-----------+-------------+-------------+--------------------+----+-----+
|Egestas Nunc Asso...|     6.58|             a|  NSW|  Male|11.28829564583802|     2299|111031231|       242936885|       6412|         6179|        12593|Books, Stationary...|2021|    8|
|Morbi Accumsan In...|     1.52|             c|  NSW|  Male|62.90176609196828|     2299|111031231|       242936885|       6412|         6179|        12593|Books, Stationary...|2021|    5|
+--------------------+---------+--------------+-----+------+

In [18]:
a.createOrReplaceTempView("agg")

temp = spark.sql(""" 

SELECT merchant_name, COUNT(merchant_name) AS no_of_transactions, SA2_code, Year, Month, SUM(dollar_value - take_rate) AS total_earnings,
    CONCAT(merchant_name, SA2_code, Year, Month) AS join_col
FROM agg
GROUP BY merchant_name, SA2_code, Year, Month
""")

temp.show()


+--------------------+------------------+---------+----+-----+------------------+--------------------+
|       merchant_name|no_of_transactions| SA2_code|Year|Month|    total_earnings|            join_col|
+--------------------+------------------+---------+----+-----+------------------+--------------------+
|  Pede Nonummy Corp.|                 6|205021084|2021|    8| 150.6670383597966|Pede Nonummy Corp...|
|    Semper Tellus PC|                 6|407031164|2021|    8| 47.51392521401466|Semper Tellus PC4...|
| Dui Nec Corporation|                 2|801041036|2021|    8|10.697061949029262|Dui Nec Corporati...|
|Ac Turpis Egestas PC|                 1|405041127|2021|    7| 158.4119995132019|Ac Turpis Egestas...|
|Lobortis Tellus C...|                 3|205021082|2021|    7|105.75408445196585|Lobortis Tellus C...|
|   Est Ac Mattis Ltd|                 1|209011204|2021|    7|            152.07|Est Ac Mattis Ltd...|
| Egestas Blandit Ltd|                 2|601051031|2021|    8|324.1176440

In [19]:
temp.createOrReplaceTempView("gender_join")
male.createOrReplaceTempView("m")
female.createOrReplaceTempView("f")

temp2 = spark.sql(""" 

SELECT *
FROM gender_join
INNER JOIN m
ON gender_join.join_col = m.m_name
""")

temp2.createOrReplaceTempView("temp2")

temp3 = spark.sql(""" 

SELECT *
FROM temp2
INNER JOIN f
ON temp2.join_col = f.f_name
""")

temp3.limit(5)

merchant_name,no_of_transactions,SA2_code,Year,Month,total_earnings,join_col,m_name,males,f_name,females
A Aliquet Ltd,2,312021351,2021,6,303.14499022452264,A Aliquet Ltd3120...,A Aliquet Ltd3120...,1,A Aliquet Ltd3120...,1
A Aliquet Ltd,2,401021010,2021,4,689.7381237115162,A Aliquet Ltd4010...,A Aliquet Ltd4010...,1,A Aliquet Ltd4010...,1
A Aliquet Ltd,2,603011065,2021,12,352.79253411685886,A Aliquet Ltd6030...,A Aliquet Ltd6030...,1,A Aliquet Ltd6030...,1
A Arcu Industries,2,124011453,2021,8,203.83657144102276,A Arcu Industries...,A Arcu Industries...,1,A Arcu Industries...,1
A Arcu Industries,2,211051282,2022,3,669.3810230931839,A Arcu Industries...,A Arcu Industries...,1,A Arcu Industries...,1


In [20]:
a = a.withColumnRenamed('income_2018-2019',

    'income_2018_2019'    
)

a = a.withColumn('income_per_persons',
    (F.col('income_2018_2019')/F.col('total_persons'))
)


In [21]:
a.show(1)

+--------------------+---------+--------------+-----+------+-----------------+---------+---------+----------------+-----------+-------------+-------------+--------------------+----+-----+------------------+
|       merchant_name|take_rate|revenue_levels|state|gender|     dollar_value|postcodes| SA2_code|income_2018_2019|total_males|total_females|total_persons|            category|Year|Month|income_per_persons|
+--------------------+---------+--------------+-----+------+-----------------+---------+---------+----------------+-----------+-------------+-------------+--------------------+----+-----+------------------+
|Egestas Nunc Asso...|     6.58|             a|  NSW|  Male|11.28829564583802|     2299|111031231|       242936885|       6412|         6179|        12593|Books, Stationary...|2021|    8|19291.422615738902|
+--------------------+---------+--------------+-----+------+-----------------+---------+---------+----------------+-----------+-------------+-------------+-----------------

In [22]:
a.createOrReplaceTempView("features")

e = spark.sql(""" 

SELECT merchant_name AS drop_name, FIRST(take_rate) AS take_rate, FIRST(revenue_levels) AS revenue_levels, FIRST(category) AS category,
    FIRST(total_males) AS males_in_SA2, FIRST(total_females) AS females_in_SA2, FIRST(income_per_persons) AS income_per_person
FROM features
GROUP BY merchant_name
""")

e.show(2)

+---------------+---------+--------------+--------------------+------------+--------------+------------------+
|      drop_name|take_rate|revenue_levels|            category|males_in_SA2|females_in_SA2| income_per_person|
+---------------+---------+--------------+--------------------+------------+--------------+------------------+
|   A Associates|     4.95|             b|Books, Stationary...|        9762|         10846|22526.523772559674|
|A Felis Company|     4.32|             b|Books, Stationary...|        1080|          1051| 33927.61168708765|
+---------------+---------+--------------+--------------------+------------+--------------+------------------+
only showing top 2 rows



In [23]:
temp3.createOrReplaceTempView("edit")
e.createOrReplaceTempView("rates")

temp4 = spark.sql(""" 

SELECT *
FROM edit
INNER JOIN rates
ON edit.merchant_name = rates.drop_name
""")

train = temp4.drop('m_name', 'f_name', 'drop_name','join_col')

train.limit(5)

merchant_name,no_of_transactions,SA2_code,Year,Month,total_earnings,males,females,take_rate,revenue_levels,category,males_in_SA2,females_in_SA2,income_per_person
A Aliquet Ltd,2,312021351,2021,6,303.14499022452264,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Aliquet Ltd,2,401021010,2021,4,689.7381237115162,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Aliquet Ltd,2,603011065,2021,12,352.79253411685886,1,1,3.87,b,Furniture,3292,3206,28693.71558221812
A Arcu Industries,2,124011453,2021,8,203.83657144102276,1,1,3.0,c,Furniture,4821,4683,25816.03452631579
A Arcu Industries,2,211051282,2022,3,669.3810230931839,1,1,3.0,c,Furniture,4821,4683,25816.03452631579


In [24]:
train.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_transactions: long (nullable = false)
 |-- SA2_code: long (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: long (nullable = false)
 |-- females: long (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- category: string (nullable = true)
 |-- males_in_SA2: long (nullable = true)
 |-- females_in_SA2: long (nullable = true)
 |-- income_per_person: double (nullable = true)



In [25]:
train_projection = train.select("merchant_name", "SA2_code", "Year", "Month", 'total_earnings')
train_projection.limit(5)

merchant_name,SA2_code,Year,Month,total_earnings
A Aliquet Ltd,401021010,2021,4,689.7381237115162
A Aliquet Ltd,603011065,2021,12,352.79253411685886
A Arcu Industries,124011453,2021,8,203.83657144102276
A Arcu Industries,211051282,2022,3,669.3810230931839
A Arcu Industries,214021379,2021,7,458.95543413409325


In [26]:
train_projection.count()

891622

In [27]:
train_projection = train_projection.withColumn("prev_year", \
              when(train_projection["Month"] == 1, train_projection['Year'] - 1).otherwise(train_projection['Year']))
train_projection = train_projection.withColumn("prev_month", \
              when(train_projection["Month"] == 1, 12).otherwise(train_projection['Month'] - 1))
train_projection = train_projection.drop("Year", "Month")
train_projection = train_projection.withColumnRenamed("total_earnings", "future_earnings") \
                            .withColumnRenamed("merchant_name", "p_merchant_name") \
                            .withColumnRenamed("SA2_code", "p_SA2_code")
train_projection.limit(5)

p_merchant_name,p_SA2_code,future_earnings,prev_year,prev_month
A Aliquet Ltd,401021010,689.7381237115162,2021,3
A Aliquet Ltd,603011065,352.79253411685886,2021,11
A Arcu Industries,124011453,203.83657144102276,2021,7
A Arcu Industries,211051282,669.3810230931839,2022,2
A Arcu Industries,214021379,458.95543413409325,2021,6


In [28]:
train_projection.count()

891622

In [29]:
final_data = train.join(train_projection, (train.merchant_name == train_projection.p_merchant_name) & 
                           (train.SA2_code == train_projection.p_SA2_code) & 
                           (train.Year == train_projection.prev_year) & 
                           (train.Month == train_projection.prev_month), how = 'inner')

final_data = final_data.drop("p_merchant_name", "p_SA2_code","prev_year", "prev_month")
final_data.limit(5)

merchant_name,no_of_transactions,SA2_code,Year,Month,total_earnings,males,females,take_rate,revenue_levels,category,males_in_SA2,females_in_SA2,income_per_person,future_earnings
A Auctor Non Corp...,3,202031033,2021,11,204.95842039835003,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,161.93542653481762
A Auctor Non Corp...,3,205021082,2022,7,260.8559102708507,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,238.45223804053688
A Auctor Non Corp...,2,205031087,2021,9,220.45304223489023,1,1,5.58,a,Furniture,2067,2014,22634.72370679088,264.8309717894243
A Auctor Non Corp...,3,210021235,2022,9,145.296601846073,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,176.47947355962134
A Auctor Non Corp...,4,211051282,2021,10,294.11572283351205,2,2,5.58,a,Furniture,2067,2014,22634.72370679088,418.80043896643065


In [30]:
final_data.count()

344019

In [31]:
final_data.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_transactions: long (nullable = false)
 |-- SA2_code: long (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: long (nullable = false)
 |-- females: long (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- category: string (nullable = true)
 |-- males_in_SA2: long (nullable = true)
 |-- females_in_SA2: long (nullable = true)
 |-- income_per_person: double (nullable = true)
 |-- future_earnings: double (nullable = true)



In [32]:
final_data = final_data.withColumn('Year',

    F.col('Year').cast('STRING')

)

final_data = final_data.withColumn('Month',

    F.col('Month').cast('STRING')

)

final_data = final_data.withColumn('SA2_code',

    F.col('SA2_code').cast('STRING')

)

field = ['no_of_transactions', 'males', 'females', 'males_in_SA2', 'females_in_SA2']

for col in field:
    final_data = final_data.withColumn(col,

    F.col(col).cast('INT')

)

In [33]:
final_data.count()

344019

In [34]:
final_data.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_transactions: integer (nullable = false)
 |-- SA2_code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: integer (nullable = false)
 |-- females: integer (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- category: string (nullable = true)
 |-- males_in_SA2: integer (nullable = true)
 |-- females_in_SA2: integer (nullable = true)
 |-- income_per_person: double (nullable = true)
 |-- future_earnings: double (nullable = true)



In [35]:
final_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final_data.columns]
   ).show()

TypeError: 'str' object is not callable

In [36]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'SA2_code', 'Year', 'Month', 'revenue_levels','category'],
outputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'], handleInvalid="keep")

indexd_data = indexer.fit(final_data).transform(final_data)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'],
outputCols = ['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec','males_in_SA2','females_in_SA2', 'income_per_person', 'no_of_transactions','take_rate', 'total_earnings'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

In [37]:
# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "future_earnings",
    "label"
)

In [38]:
# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [39]:
# Split the data into training and validation sets (30% held out for testing)

trainingData, testData = outdata1.randomSplit([0.7, 0.3], seed = 20)

In [40]:
trainingData.count(), testData.count()

22/10/05 01:26:49 WARN DAGScheduler: Broadcasting large task binary with size 1333.0 KiB


22/10/05 01:27:49 WARN DAGScheduler: Broadcasting large task binary with size 1333.8 KiB


(240534, 103485)

In [41]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")


# Train model.  
model = rf.fit(trainingData)

# Make predictions.
predictions_validation = model.transform(testData)

22/10/05 01:28:50 WARN DAGScheduler: Broadcasting large task binary with size 1336.0 KiB


22/10/05 01:28:51 WARN DAGScheduler: Broadcasting large task binary with size 1336.0 KiB


22/10/05 01:28:53 WARN DAGScheduler: Broadcasting large task binary with size 1340.0 KiB


22/10/05 01:28:55 WARN DAGScheduler: Broadcasting large task binary with size 1452.6 KiB


22/10/05 01:29:01 WARN DAGScheduler: Broadcasting large task binary with size 1514.3 KiB


22/10/05 01:29:05 WARN DAGScheduler: Broadcasting large task binary with size 1635.3 KiB


22/10/05 01:29:08 WARN DAGScheduler: Broadcasting large task binary with size 1871.5 KiB


22/10/05 01:29:12 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB


In [42]:
# Evaluate the validation set 

predictions_validation.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error

evaluator_train_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse_train = evaluator_train_rmse.evaluate(predictions_validation)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_train)

evaluator_train_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae_train = evaluator_train_mae.evaluate(predictions_validation)
print("Mean Absolutee Error (MAE) on train data = %g" % mae_train)

22/10/05 01:30:16 WARN DAGScheduler: Broadcasting large task binary with size 1348.1 KiB


+------------------+------------------+--------------------+
|        prediction|             label|            features|
+------------------+------------------+--------------------+
|329.41821754490013| 163.5578782258266|(2085,[209,1093,2...|
| 348.3083657034912|161.93542653481762|(2085,[209,988,20...|
| 348.3083657034912| 267.4761652214389|(2085,[209,976,20...|
|375.37587840988203|418.80043896643065|(2085,[209,989,20...|
| 344.8823175397339| 145.1093154066284|(2085,[209,1039,2...|
+------------------+------------------+--------------------+
only showing top 5 rows



22/10/05 01:31:18 WARN DAGScheduler: Broadcasting large task binary with size 1341.3 KiB


22/10/05 01:31:20 WARN DAGScheduler: Broadcasting large task binary with size 1342.4 KiB
Root Mean Squared Error (RMSE) on train data = 341.445


22/10/05 01:32:19 WARN DAGScheduler: Broadcasting large task binary with size 1341.2 KiB


22/10/05 01:32:21 WARN DAGScheduler: Broadcasting large task binary with size 1342.3 KiB
Root Mean Squared Error (MAE) on train data = 205.915


In [43]:
latest_year = train.select(max('Year')).collect()[0][0]
agg_month_1 = train.filter(train.Year == latest_year)
latest_month = agg_month_1.select(max('Month')).collect()[0][0]
predicting_data = agg_month_1.filter(train.Month == latest_month)
predicting_data = predicting_data.withColumn("future_earnings", lit(0))
predicting_data.limit(5)

merchant_name,no_of_transactions,SA2_code,Year,Month,total_earnings,males,females,take_rate,revenue_levels,category,males_in_SA2,females_in_SA2,income_per_person,future_earnings
A Auctor Non Corp...,3,125031480,2022,10,200.96402947067224,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,0
A Auctor Non Corp...,3,126011496,2022,10,200.92855368870426,1,2,5.58,a,Furniture,2067,2014,22634.72370679088,0
A Auctor Non Corp...,3,211051282,2022,10,195.89763542546956,2,1,5.58,a,Furniture,2067,2014,22634.72370679088,0
A Auctor Non Corp...,2,509021239,2022,10,101.52780947033744,1,1,5.58,a,Furniture,2067,2014,22634.72370679088,0
A Auctor Non Corp...,6,509021240,2022,10,350.12284979054124,3,2,5.58,a,Furniture,2067,2014,22634.72370679088,0


In [44]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'SA2_code', 'Year', 'Month', 'revenue_levels','category'],
outputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'], handleInvalid="keep")

indexd_data = indexer.fit(predicting_data).transform(predicting_data)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'SA2_code_num', 'Year_num', 'Month_num', 'revenue_levels_num','category_num'],
outputCols = ['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['merchant_name_vec', 'SA2_code_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec','category_vec','males_in_SA2','females_in_SA2', 'income_per_person', 'no_of_transactions','take_rate', 'total_earnings'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "future_earnings",
    "label"
)


# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [45]:
predictions_test = model.transform(outdata1)

In [49]:
predictions_test.show()

22/10/05 01:44:23 WARN DAGScheduler: Broadcasting large task binary with size 1190.3 KiB
+--------------------+------------------+---------+----+-----+------------------+-----+-------+---------+--------------+--------------------+------------+--------------+------------------+-----+-----------------+------------+--------+---------+------------------+------------+------------------+------------------+-------------+-------------+------------------+-------------+--------------------+--------------------+------------------+
|       merchant_name|no_of_transactions| SA2_code|Year|Month|    total_earnings|males|females|take_rate|revenue_levels|            category|males_in_SA2|females_in_SA2| income_per_person|label|merchant_name_num|SA2_code_num|Year_num|Month_num|revenue_levels_num|category_num| merchant_name_vec|      SA2_code_vec|     Year_vec|    Month_vec|revenue_levels_vec| category_vec|            features|     indexedFeatures|        prediction|
+--------------------+---------------

In [50]:
predictions_test.createOrReplaceTempView("preds")

pred = spark.sql(""" 

SELECT merchant_name, SUM(prediction) AS total_revenue
FROM preds
GROUP BY merchant_name

""")

pred.limit(5)

merchant_name,total_revenue
Dictum Mi Incorpo...,558.7125016111587
Dictum Mi Limited,15085.237543501285
Donec Luctus Indu...,3910.987511278111
Elit Sed Consequa...,12850.387537056651
Hendrerit Consect...,2514.206257250214


In [55]:
pred.count()

1381

In [52]:
pred_df = pred.toPandas()

In [54]:
pred_df.to_csv("../data/curated/revenue.csv")